**Summary:** The following notebook contains the code to solve the DCJ-median problem using a genetic algorithm metaheuristic approach. The code contains functions to perform DCJ operations on genomes, construct the adjacency graph between two genomes and use it to determine the distance between them, read in the data files and translate to our genome representation, perform crossover and mutations, generate an initial population, find DCJ operations on the sorting path from one genome to another, and find a median genome using a genetic algorithm approach incorporating all the aforementioned functions. Our implementation is inteded to find the median ancestral genome for lions, tigers, and snow leopards. Our read file function takes the tsv files from GenBank and preprocesses them for use in the notebook. Attatched in our submission is the sample simulated genome data included with our reference paper. A second function for reading in that data has been included as well. We performed multiple experiments using our big cat genome data, the reference paper's data, and our own simulated data along with varying values for parameters such as max_generations and per_step. Ultimtely, we found our algorithm to be effective, but can suffer from runtime issues due to complex computations arising from operations such as finding DCJ operations on a sorting path between two genomes and creating the adjacency graphs.

**Material and Methods:** In this section, we present how we developed our algorithm using a genetic heuristic and verified its accuracy with small verifiable, manufactured data sets. We also show how to prepare genomic data from NCBI for use in our algorithm and illustrate how to tackle major problems that can arise.
Background- The DCJ median problem
The DCJ median problem involves finding a common ancestor or median genome that minimizes the sum of evolutionary events: inversions, translocations, fissions, and fusions are required to transform a set of input genomes into the median genome (Yancopoulous et al., 2005; Gao et al., 2013). Our algorithm takes three genomes of modern species and attempts to follow the possible rearrangements of each species genome to recreate the ancestral genome. The median of the three genomes is thought to represent the ancestral species. If the distances between three possible genomes were represented as a triangle, the median genome would be the centroid of the triangle (Figure 1).

![dcj_triangle.jpeg](attachment:dcj_triangle.jpeg)

Figure 1: This figure is a basic representation of the DCJ median problem. Three genomes are placed to create a triangle. The dark line along the edge of the triangle represents the edit distance between the two genomes it connects. In the center of the triangle, there is a black dot, representing the median of all three genomes. The dashed line moving away from the corners of the genomes represents the shortest distance for the genome to get to the median. The colored paths represent the wandering path that the genetic algorithm takes as the genomes approach the median.

In DCJ, genomes are represented as signed permutations, where each gene corresponds to a signed integer indicating its orientation on a chromosome (Sturtevant and Novitski, 1941; Sankoff, 1992). The algorithm then iteratively applies DCJ operations to shuffle the gene order, aiming to converge towards a median genome that minimizes the sum of events across all input genomes (Gao et al., 2005). The optimization process continues until a stable median genome is reached, offering insights into the evolutionary history shared among the input genomes.
Consider the genome G with a signed ordering of genes g₁, g₂, ..., gₙ. There are four main types of rearrangements the genome can undergo. A reversal operation between indices i and j (where i ≠ j) produces a new genome with a linear ordering of genes g₁, g₂, ..., gᵢ₋₁, gⱼ, gⱼ₋₁, ..., gᵢ, gⱼ₊₁, ..., gₙ. For genomes with more than one chromosome, additional events come into play. These include translocation, where the end of one chromosome is broken and attached to the end of another; fission, where one chromosome splits into two; and fusion, where two chromosomes combine into one. The “double-cutand-join” (DCJ) model explains each genome rearrangement event as a single DCJ operation and is mathematically simpler than handling each event individually.
A method of finding shorter paths of rearrangement steps is to use an adjacency graph. Genes i and j are considered adjacent if i is followed by j or -j is followed by -i. The notion of a breakpoint arises when two genes are adjacent in one genome but not in the other. Telomeres at the ends of chromosomes lack adjacency with any other genes.
When the genomes undergo DCJ, they are cut at adjacencies of indicated breakpoints. The order of the cuts is indicated by an adjacency graph that analyzes to find the shortest path for improving its similarity to the other genomes.


![dcj_adjacency.jpeg](attachment:dcj_adjacency.jpeg)


Figure 3: Agency graph. The graph represents four cycles. The bottom row (in blue) represents the target genome. In each iteration, the section of the genome that undergoes the DCJ operation is indicated, rearranging the genome into the target genome.
Background-Genetic Algorithms
Genetic algorithms were first developed in the 1960s by John Holland and his collaborators but didn’t start to become popular until the late 1980s, when programmers started utilizing them to evolve and adapt their code (Holland, 1975; Koza JR, 1990). Genetic algorithms (GAs) are optimization techniques inspired by the principles of natural selection. Beginning with an initial population of potential solutions represented as individuals with sets of parameters, GAs iteratively evolve solutions to complex optimization problems. The process involves evaluating the fitness of each individual based on a predefined fitness function, selecting individuals for the next generation based on their fitness, and applying genetic operations such as crossover (recombination) and mutation to generate offspring. Crossover mimics genetic recombination, while mutation introduces random changes, fostering diversity. The next generation is formed by combining the individuals with highest “fitness scores” and their offspring. This process continues until termination criteria are met. GAs are particularly effective for problems with large solution spaces and complex landscapes, offering a heuristic approach for finding approximate solutions over successive generations.
 Our approach:
The GenoCATalyst algorithm is used to find the ancestral genome that is represented by the median point of three genomes. Our approach combined the idea of the DCJ-median problem with a genetic algorithm to imitate real life genome rearrangement events, and a new generation is selected according to their fitness. GenoCATalyst uses a dynamic programming approach to calculate the fitness score for each genome in each iteration of the algorithm. It utilizes an adjacency graph to reduce the distance between each genome and puts forth the best possible candidates each round to estimate closer and closer to the median ancestor.
Data collection and pre-processing
Representative species genomes used in our data analysis were obtained from NCBI’s genome library. Species genomes were selected based on the availability of chromosome information and NCBI reference genome validation as reference genomes. With the wide availability of sequencing possible today, many species have been sequenced, but not all assemblies provide details on gene orientation or position on chromosomes. The genomes we selected include the following in the big cat family: lion, tiger, snow leopard (panthera leo, panthera tigris, panthera unica). All three of these are in the Panthera genus, and thus reduce the distance of differences between the potential genomic profiles. Their last common ancestor was ~3.5 million years ago (O’Brian, 2007)
I. Developing the algorithm
Individuals of the initial population have a deep impact on the performance of a genetic-algorithm based approach. In a similar method of Gao et al (2013), we generated our starting population based on the following observation that given three genomes, the median path is more likely to be on one leaf than the other (Moret et al, 2002). We first look at the distances between the three base genomes, and the larger the distance, the larger of an initial population we generate to cover a larger search space. For each pair of genomes, we iterate over distance/10, 2*(distance/10), … ,6*(distance/10) to define the number of steps. We then generate a ‘per_step’ number of genomes on the sorting path from the first genome in the pair, where ‘per_step’ is a variable passed into the initial population generation function that we can define and fine tune ourselves. Genomes are generated on the sorting path by applying random DCJ operations. This is repeated until all three genome pairs have been looped through and results in our starting population. Generating the starting population in this method helps to improve the likelihood that the algorithm will reach conversion (Gao et al 2013). In testing, we attempted to generate enough diversity by generating 50 genomes per sampled step.
 Fine tuning the performance of the algorithm involved utilizing the premade Julia functions over loop creations, as the Julia functions were already optimized to operate in the language. During much of the modification of the adjacency graph. Further optimization of the algorithm involved modifying the number of ‘max generations’ that the algorithm went through before termination. The greater the size of ‘max generations’, the longer the algorithm can run, especially if it never converges. Other variables that could be modified for speed vs accuracy trade off include changing the amount of DCJ operations performed in the crossover and mutate functions. Lowering the amount of operations in those functions increases the speed of the algorithm's main loop, at the cost of having a slower rate of convergence. For testing purposes, we also ran the code a few times using the hamming distance to calculate the median score for the fitness function rather than the DCJ distance. This was just to improve the runtime of the algorithm, as the hamming distance is less computationally intensive, and was used mainly to verify the code was running properly elsewise.
II. Output/visualization We output a graph of fitness score over number of iterations (Figure 5). This helps the user to see that the algorithm is progressing towards convergence as more iterations occur.

![plot.png](attachment:plot.png)

Figure 5: Expected output graph illustrating decreasing fitness score over iteration as the algorithm approaches convergence.
III. Testing the algorithm We tested its validity by using our DCJ function on a smaller set of simulated data to ensure that it behaved as expected. To compare performance between algorithms, the same simulated genomes were run in GenoCATalyst and ASMedian algorithms. The genomes were simulated with increasing size (quantity of genes) and increasing number of reversals (percent mutated) in order to evaluate how well the algorithms preformed in terms of time, memory and accuracy.

**Results:** Our experiments thereby began by determining how genome size affected different performance metrics of the algorithm, such as time, memory and score. We used the ASMedian algorithm developed by Sankoff and Banette (1998) to compare our algorithm’s performance. GenoCATalyst and ASMedian performance was compared over size of genomes (number of genes) for the median score it produced (Figure 6A. B) and compared to the difference between GenoCATalyst and ASMedian scores (Figure 6C)

Figure 6: Comparison of the reported median score in [our algorithm] and ASMedian. Top chart depicts GenoCATalyst middle chart depicts the reported median score for the ASMedian algorithm. Colors indicate genome size tested. We also tested and compared the quantity of memory consumed by GenoCATalyst. The results can be seen in figure 7.

![dcj_memory.png](attachment:dcj_memory.png)

Figure 7: Comparison of GenoCATalyst memory use (MB) vs ASMedian algorithm. Performance times were also compared between GenoCATalyst as compared to the exact ASMedian solver (Figure 8). The ASM median solver has time performance issues when greater than 40% of the genome is mutated, as seen in figure 8a. Figure 8b illustrates the differences in runtime between GenoCATalyst and ASMedian solver when the genetic differences between ancestral and modern genomes are less than 40%

![dcj_time_diff.png](attachment:dcj_time_diff.png)

Figure 8: Comparison of the difference of GenoCATalyst vs ASMedian time performance.

**Discussion:** In this paper, we discussed an algorithm that attempted to reconstruct the ancestral genome of several different large cat species. We used an algorithmic approach for the DCJ median problem, along with a genetic algorithm as a heuristic approach. Ultimately, GenoCATalyst did not perform as we hoped it would. Given more time and resources, we would have tried to improve the algorithm through parallelization of the code so that it would be able to run on multiple cores for improved performance. Ideally that would enable us to estimate the ancestor of lions, tigers, and snow leopards as we had originally intended.
When we were finally set up to run our algorithm on the big cat genomic data, we were suddenly faced with the fact that we did not have enough time to process it. The adjacency graphs for each pair of genomes were taking tens of minutes to complete, and we would have to be creating many per generation iteration that the algorithm ran. We continued to try to assess our algorithm with the data permutations from the Gao et al (2013) paper, as well as from simulations of reversions.
Our algorithm did find the DCJ distance of our test sequences accurately. Also, the median score calculated for our algorithm vs the ASMedian algorithm were consistent up to point where the ASMedian algorithm starts becoming less efficient (when analyzing more distant genomes). Given more time, we would be hopeful to get our algorithm fully functional.
Reconstruction of the ancestral genome can give insights into comparative genetics, which can allow for comparison of the genetic makeup of ancestral species with their modern descendants. This is important to showing how genetics have changed over time and aid in understanding of evolutionary relationships. Additionally, There are a few applications that an ancestral genome could provide to modern conservation efforts. Knowledge of the ancestral genome can help biologists recognize genetic trends that have been preserved in the organism’s history. Understanding the ancestral genome can be utilized in designing conservation strategies to preserve genetic heritage. Conservation plans can be developed to maintain historically significant genes while planning breeding programs to help conserve and maintain endangered species. Knowledge of the ancestral genome could assist geneticists and conservationists to create a gene therapy plan to replace faulty genes with better versions from the past. Someday, solving for the ancestral genome may be the key to maintaining genetic diversity of life on our planet.  

**References:**

Gao N, Yang N, Tang J (2013) Ancestral genome inference using a genetic algorithm approach. PLOS ONE 8(5): e62156. https://doi.org/10.1371/journal.pone.0062156 [date accessed: 10/23/2023]
Holland, J. H. (1975). Adaptation in natural and artificial systems: An introductory analysis with applications to biology, control, and artificial intelligence. U Michigan Press.
Koza JR. Genetic programming: A paradigm for genetically breeding populations of computer programs to solve problems. Stanford, CA: Stanford University, Department of Computer Science; 1990 Jun 1.
 Moret, B.M.E., Siepel, A.C., Tang, J., Liu, T. (2002). Inversion Medians Outperform Breakpoint Medians in Phylogeny Reconstruction from Gene-Order Data. In: Guigó, R., Gusfield, D. (eds) Algorithms in Bioinformatics. WABI 2002. Lecture Notes in Computer Science, vol 2452. Springer, Berlin, Heidelberg. https://doi.org/10.1007/3-540-45784-4_40
O’Brien, J. S. and Johnson, W. E. (2007). The evolution of cats. Scientific American July 2007: 68-75
Rajan, Vaibhav et al. “Heuristics for the inversion median problem.” BMC bioinformatics vol. 11 Suppl 1,Suppl 1 S30. 18 Jan. 2010, doi:10.1186/1471-2105-11-S1-S30
Rubert, D.P., Feijão, P., Braga, M.D.V. et al. Approximating the DCJ distance of balanced genomes in linear time. Algorithms Mol Biol 12, 3 (2017). https://doi.org/10.1186/s13015-017-0095-y
Sankoff, D. Edit distance for genome comparison based on non-local operations. In Apostolico, A., Crochemore, M., Galil, Z., Manber, U. (Eds.). Combinatorial Pattern Matching. Third Annual Symposium, Lecture Notes in Computer Science. 1992. Springer Verlag 644, pp. 121–135.
 Sankoff D, Blanchette M (1998) Multiple genome rearrangement and breakpoint phylogeny. Journal of Computational Biology 5: 555–570.
Sturtevant, A H, and E Novitski. The Homologies of the Chromosome Elements in the Genus Drosophila. Genetics vol. 26,5 (1941): 517-41. doi:10.1093/genetics/26.5.517 (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1209144/pdf/517.pdf) [date accessed: 11/19/2023]
Xu W, Sankoff D (2008). Decompositions of multiple breakpoint graphs and rapid exact solutions to the median problem. In: 8th International Workshop on Algorithms in Bioinformatics (WABI 2008). pp. 25-37.
Yancopoulos S, Attie O, Friedberg R, Efficient Sorting of Genomic Permutations by Translocation, Inversion and Block Interchange, Bioinformatics, Volume 21, Issue 16, August 2005, Pages 3340–3346, https://doi.org/10.1093/bioinformatics/bti535 [date accessed: 10/29/2023]


**DCJ Operation:**

The crux of the entire process, the dcj_operation function performs a dcj operation on an input genome, and returns the resulting genome. The function takes in four indices, two for whcih chromosomes, and two for the genes in those respective chromosomes. The function is able to handle interchromosomal and intrachromosomal operations. The function can perform operations on both linear and circular chromosomes, by first checking for circulairty in each. The function aslo checks for telomeres in order to appropriately decide if the operation will be one of the following:

-Fission

-Inversion

-Fusion

-Translocation

In [ ]:
function dcj_operation(genome, index1, chromosome1, index2, chromosome2)
    # Ensure indices are within bounds
    if chromosome1 > length(genome) || chromosome2 > length(genome) ||
       index1 > length(genome[chromosome1]) || index2 > length(genome[chromosome2])
        error("Index out of bounds")
    end

    # Swap indices and chromosomes if necessary for consistency
    if index1 > index2
        index1, index2 = index2, index1
        chromosome1, chromosome2 = chromosome2, chromosome1
    end

    # Clone the genome array to avoid mutating the original
    new_genome = deepcopy(genome)

    # Check for circularity
    is_circular1 = !in(0, new_genome[chromosome1])
    is_circular2 = !in(0, new_genome[chromosome2])

    if chromosome1 == chromosome2
        # Handling the same chromosome
        part1 = new_genome[chromosome1][1:index1-1]
        part2 = new_genome[chromosome1][index1:index2]
        part3 = new_genome[chromosome1][index2+1:end]

        if is_circular1
            # Handling circular chromosome
            new_genome[chromosome1] = vcat(part1, reverse(part2), part3)
        else
            # Handling linear chromosome
            if new_genome[chromosome1][index1] == 0 || new_genome[chromosome1][index2] == 0
                # Fission - correctly place telomeres at the ends
                part2_start = index1 + (new_genome[chromosome1][index1] == 0 ? 1 : 0)
                part2 = new_genome[chromosome1][part2_start:index2-1]
                if !isempty(part2)
                    new_genome[chromosome1] = vcat(part1, 0)
                    push!(new_genome, vcat(0, reverse(part2), part3))
                end
            else
                # Inversion
                new_genome[chromosome1] = vcat(part1, reverse(part2), part3)
            end
        end
    else
        # Handling different chromosomes
        part1 = new_genome[chromosome1][1:index1-1]
        part2 = new_genome[chromosome1][index1:end]
        part3 = new_genome[chromosome2][1:index2-1]
        part4 = new_genome[chromosome2][index2:end]

        if is_circular1 || is_circular2
            # Handling circular chromosomes
            new_genome[chromosome1] = vcat(part1, part4)
            new_genome[chromosome2] = vcat(part3, part2)
        else
            # Handling linear chromosomes
            if new_genome[chromosome1][index1] == 0 && new_genome[chromosome2][index2] == 0
                # Fusion - join chromosomes without telomeres in between
                if length(part1) > 1 && length(part4) > 1
                    new_genome[chromosome1] = vcat(part1, part4[2:end])
                    new_genome[chromosome2] = []
                elseif length(part2) > 1 && length(part3) > 1
                    new_genome[chromosome1] = vcat(part3[1:end-1], part2)
                    new_genome[chromosome2] = []
                end
            else
                # Translocation
                new_genome[chromosome1] = vcat(part1, part4)
                new_genome[chromosome2] = vcat(part3, part2)
            end
        end
    end

    # Remove any empty chromosomes
    new_genome = filter(chromosome -> !isempty(chromosome), new_genome)

    return new_genome
end


dcj_operation (generic function with 1 method)

**Adjacency Graph:**

In order to properly determine the dcj distance between two genomes, the corresponding adjacency graph has to be constucted. This function creates a graph using the LightGraphs package and adds edges between the adjacencies in each genome. This function can be quite computationally expensive when having to handle large inputs, and has proven to be the main bottleneck in running the code.

In [ ]:
using LightGraphs

function adjacency_graph(genome1, genome2)
    # Function to create adjacency pairs from a chromosome
    function get_adjacency_pairs(chromosome)
        adjacencies = []
        n = length(chromosome)
        is_circular = chromosome[1] != 0 && chromosome[end] != 0

        if is_circular
            push!(adjacencies, (chromosome[end], -chromosome[1]))
        elseif chromosome[1] != 0
            push!(adjacencies, (0, chromosome[1]))
        end

        for i in 1:(n - 1)
            push!(adjacencies, (chromosome[i], -chromosome[i + 1]))
        end

        if !is_circular && chromosome[end] != 0
            push!(adjacencies, (chromosome[end], 0))
        end

        return adjacencies
    end

    # Collect all adjacencies from both genomes
    adjacencies_genome1 = [get_adjacency_pairs(chromosome) for chromosome in genome1]
    adjacencies_genome2 = [get_adjacency_pairs(chromosome) for chromosome in genome2]

    # Flatten the lists for easier processing
    flat_adjacencies_genome1 = vcat(adjacencies_genome1...)
    flat_adjacencies_genome2 = vcat(adjacencies_genome2...)

    # Create vertices and initialize graph
    total_adjacencies = vcat(flat_adjacencies_genome1, flat_adjacencies_genome2)
    #sg = SimpleGraph(length(total_adjacencies))
    #g = MetaGraph(length(total_adjacencies))
    g = SimpleDiGraph(length(total_adjacencies))

    # Create a mapping from adjacencies to vertex indices
    vertex_map_genome1 = Dict(adjacency => i for (i, adjacency) in enumerate(flat_adjacencies_genome1))
    offset = length(flat_adjacencies_genome1)
    vertex_map_genome2 = Dict(adjacency => i + offset for (i, adjacency) in enumerate(flat_adjacencies_genome2))

    # Add edges based on shared non-telomere elements
    for adjacency1 in flat_adjacencies_genome1
        index1 = vertex_map_genome1[adjacency1]
        for adjacency2 in flat_adjacencies_genome2
            index2 = vertex_map_genome2[adjacency2]

            if adjacency1[1] != 0 && (adjacency1[1] == adjacency2[1] || adjacency1[1] == adjacency2[2])
                add_edge!(g, index1, index2)
            end
            if adjacency1[2] != 0 && (adjacency1[2] == adjacency2[1] || adjacency1[2] == adjacency2[2])
                add_edge!(g, index2, index1)
            end
        end
    end

    return g, vertex_map_genome1, vertex_map_genome2
end


adjacency_graph (generic function with 1 method)

Example vertex mappings and edges from the adjacency graph following the example genomes from the slides

In [ ]:
A = [[0, 1, -2, 3, 4, 0],[5, 6, 7]]
B = [[0, 1, 2, 3, 0],[0, 4, 5, 6, 7, 0]]

graph, vertex_map1, vertex_map2 = adjacency_graph(A, B)

println(vertex_map1)
println(vertex_map2)
for e in edges(graph)
    println(e)
end


Dict((4, 0) => 5, (1, 2) => 2, (5, -6) => 7, (0, -1) => 1, (7, -5) => 6, (6, -7) => 8, (3, -4) => 4, (-2, -3) => 3)
Dict((1, -2) => 10, (5, -6) => 15, (7, 0) => 17, (0, -1) => 9, (4, -5) => 14, (2, -3) => 11, (6, -7) => 16, (3, 0) => 12, (0, -4) => 13)
Edge 2 => 10
Edge 3 => 10
Edge 4 => 12
Edge 5 => 14
Edge 6 => 17
Edge 7 => 15
Edge 8 => 16
Edge 9 => 1
Edge 11 => 2
Edge 11 => 3
Edge 13 => 4
Edge 14 => 6
Edge 15 => 7
Edge 16 => 8


**Get Operations:**

This function returns genome that is max_operations on the sorting path from the source genome to the target genome. This is used in the crossover and mutate portions of the genetic algorithm following the process described in the reference paper. This function is an adaptation of the pseudocode for the DCJSORT algorithm discussed in class. This function has also proved to be a bottleneck when running the code, so different ranges of values for max_operations can be used to speed up the overall runtime with the tradeoff of slowing down the rate of convergence.

In [ ]:
function get_operations(source, target, max_operations)
    modified_source = deepcopy(source)
    operation_count = 0

    while operation_count < max_operations
        operation_found = false

        # Process non-telomere adjacencies in target genome
        for t_chr_idx in 1:length(target)
            for t_gene_idx in 1:(length(target[t_chr_idx]) - 1)
                p = target[t_chr_idx][t_gene_idx]
                q = target[t_chr_idx][t_gene_idx + 1]

                if p != 0 && q != 0
                    u_chromosome = v_chromosome = -1
                    u_index = v_index = -1

                    # Find positions of p and q in modified_source genome
                    for s_chr_idx in 1:length(modified_source)
                        for s_gene_idx in 1:(length(modified_source[s_chr_idx]) - 1)
                            if modified_source[s_chr_idx][s_gene_idx] == p
                                u_chromosome, u_index = s_chr_idx, s_gene_idx
                            end
                            if modified_source[s_chr_idx][s_gene_idx + 1] == q
                                v_chromosome, v_index = s_chr_idx, s_gene_idx + 1
                            end
                        end
                    end

                    if u_chromosome != -1 && v_chromosome != -1
                        # Apply the operation
                        modified_source = dcj_operation(modified_source, u_index, u_chromosome, v_index, v_chromosome)
                        operation_found = true
                        operation_count += 1
                        break  # Break to recompute the adjacency graph
                    end
                end
            end
            if operation_found
                break
            end
        end

        if !operation_found
            # Process telomeres in the target genome
            for t_chr_idx in 1:length(target)
                for t_gene_idx in [1, length(target[t_chr_idx])]
                    p = target[t_chr_idx][t_gene_idx]
                    if p != 0
                        u_chromosome = -1
                        u_index = -1

                        # Find the chromosome in modified_source with the corresponding gene
                        for s_chr_idx in 1:length(modified_source)
                            for s_gene_idx in 1:length(modified_source[s_chr_idx])
                                if modified_source[s_chr_idx][s_gene_idx] == p
                                    u_chromosome, u_index = s_chr_idx, s_gene_idx
                                    # Apply the operation
                                    modified_source = dcj_operation(modified_source, u_index, u_chromosome, u_index, u_chromosome)
                                    operation_count += 1
                                    operation_found = true
                                    break
                                end
                            end
                            if operation_found
                                break
                            end
                        end
                    end
                    if operation_found
                        break
                    end
                end
            end
        end

        if !operation_found
            # No more operations can be found
            break
        end
    end

    return modified_source
end


get_operations (generic function with 1 method)

**DCJ Distance:**

One crucial aspect is making sure the dcj distance is properly computed so that the fitness function can provide an accurate score. This function follows the dcj distance equation can looks for the number of cycles and odd-edge paths in the adjacency graph between the two genomes.

In [ ]:
using LightGraphs

function dcj_distance(genome1, genome2)
    g, _ = adjacency_graph(genome1, genome2)
    components = connected_components(g)

    C = 0  # Number of cycles of length 2
    I = 0  # Number of odd edge paths

    for component in components
        # Check if the component forms a cycle of length 2
        if all(v -> degree(g, v) == 2, component)
            C += 1
        else
            I += 1
        end
    end

    n = sum(count(x -> x != 0, chromosome) for chromosome in genome1)
    return n - (C + I ÷ 2)
end




dcj_distance (generic function with 1 method)

**Initial Population:**

Generates the initial population to be used in the genetic algorithm. The genomes are generated on a random distance from thee genomes we're finding the median for. The size of the initial population varies depending on the distance between the three genomes.

In [ ]:
function initial_population(genomes, per_step)
    population = []

    # Calculate DCJ distances between all pairs of genomes
    distances = [dcj_distance(genomes[1], genomes[2]),
                 dcj_distance(genomes[1], genomes[3]),
                 dcj_distance(genomes[2], genomes[3])]

    # Iterate over all pairs of genomes
    j = 1
    for i in 1:3
        genome_pair = i != 3 ? (genomes[1], genomes[i+1]) : (genomes[2], genomes[3])

        # Calculate steps and iterate over them
        for step in ceil(Int, distances[i]/10):ceil(Int, distances[i]/10):floor(Int, 6*distances[i]/10)
            # Generate per_step genomes for each step
            for _ in 1:per_step
                new_genome = deepcopy(genome_pair[1])
                # Apply DCJ operations step times to generate a new genome
                for _ in 1:step
                    chromosome_index1 = rand(1:length(new_genome))
                    gene_index1 = rand(1:length(new_genome[chromosome_index1]))

                    # Ensure the second set of indices is different from the first
                    chromosome_index2, gene_index2 = chromosome_index1, gene_index1
                    while chromosome_index2 == chromosome_index1 && gene_index2 == gene_index1
                        chromosome_index2 = rand(1:length(new_genome))
                        gene_index2 = rand(1:length(new_genome[chromosome_index2]))
                    end
                    new_genome = dcj_operation(new_genome, gene_index1, chromosome_index1, gene_index2, chromosome_index2)
                end
                println(j)
                j = j + 1
                push!(population, new_genome)
            end
        end
    end
    return population
end


initial_population (generic function with 1 method)

Finds the perfect median score of the three genomes based off the equation given in the reference paper

In [ ]:
function best_score(genome1, genome2, genome3)
    distance12 = dcj_distance(genome1, genome2)
    distance13 = dcj_distance(genome1, genome3)
    distance23 = dcj_distance(genome2, genome3)

    return ceil((distance12 + distance13 + distance23)/2)
end

best_score (generic function with 1 method)

Hamming distance function used for testing purposes

In [ ]:
function hamming_distance(genome1, genome2)
    max_chromosomes = max(length(genome1), length(genome2))
    total_distance = 0

    for i in 1:max_chromosomes
        chromo1 = i <= length(genome1) ? genome1[i] : []
        chromo2 = i <= length(genome2) ? genome2[i] : []
        max_length = max(length(chromo1), length(chromo2))

        for j in 1:max_length
            if j > length(chromo1) || j > length(chromo2) || chromo1[j] != chromo2[j]
                total_distance += 1
            end
        end
    end

    return total_distance
end


hamming_distance (generic function with 1 method)

**Median Score:**

Calculates the median score of the genome against the three genomes we're finding the median for. This function is essential for finding the fitness score of each genome in the population. However, due to how complex computing the dcj distance can be, this function can become a major bottleneck, and a large amount of time can be spent on repeated calls to this function on every iteration. As such, a supplemental version is supplied in the commented out portion that computes using the hamming distance rather than the dcj distance. This does not provide an accurate score, but can be used to speed up the overall runtime to verify that the genetic algorithm works, and as such is mainly used for testing purposes.

In [ ]:
using Statistics

function median_score(genome1, genome2, genome3, m)
    distance1 = dcj_distance(genome1, m)
    distance2 = dcj_distance(genome2, m)
    distance3 = dcj_distance(genome3, m)

    # Calculate the median of the three distances
    return median([distance1, distance2, distance3])
end

#=
using Statistics

function median_score(genome1, genome2, genome3, m)
    distance1 = hamming_distance(genome1, m)
    distance2 = hamming_distance(genome2, m)
    distance3 = hamming_distance(genome3, m)

    # Calculate the median of the three distances
    return median([distance1, distance2, distance3])
end
=#

median_score (generic function with 1 method)

**Fitness:**

This function returns the fitness score of the genome by implementing the equation (N-(S-Sbest)) given in the reference paper

In [ ]:
function fitness(genome1, genome2, genome3, m, best)
    #N = count(x -> x > 0, m)
    N = sum(count(x -> x > 0, chromosome) for chromosome in m)
    S = median_score(genome1, genome2, genome3, m)

    return N - (S - best)
end

fitness (generic function with 1 method)

**Get Scores:**

Returns the entire population sorted by their fitness scores in descending order for processing purposes in the genetic algorithm

In [ ]:
function get_scores(genomes, pop, best)
    #fit_dict = Dict{Vector{Int64}, Int}()
    fit_dict = Dict{Vector{Vector{Int}}, Int}()

    for m in pop
       fit_dict[m] = fitness(genomes[1], genomes[2], genomes[3], m, best)
        print('-')
        flush(stdout)
    end

    # Sort the dictionary by values in descending order and return the keys
    sorted_genomes = [kv[1] for kv in sort(collect(fit_dict), by = x -> x[2], rev = true)]
    println()

    return fit_dict, sorted_genomes
end




get_scores (generic function with 1 method)

**Crossover:**

Crossover creates two child genomes from two parent genomes to be added to the new population. The fitness scores of the two parents are measured in order to determine a better parent and a worse parent. The first child, C1, is a direct copy of the better parent, while the second child, C2, is generated on the sorting path between the worse parent and the better parent.

In [ ]:
function crossover(parent1, parent2, genome1, genome2, genome3, best)
    # Calculate fitness scores for both parents
    F1 = fitness(genome1, genome2, genome3, parent1, best)
    F2 = fitness(genome1, genome2, genome3, parent2, best)

    # Determine which parent is better
    better_parent, worse_parent = F2 < F1 ? (parent2, parent1) : (parent1, parent2)

    # Initialize the first child (C1) as a copy of the worse parent
    C1 = deepcopy(worse_parent)

    # Randomly determine the number of operations to apply
    #m = rand(1:min(50, maximum([length(chromosome) for chromosome in better_parent])))
    m = rand(1:5)
    C1 = get_operations(C1, better_parent, m)
    C2 = deepcopy(better_parent)

    return C1, C2
end



crossover (generic function with 1 method)

**Mutate:**

Mutation occurs in the population to induce some degree of random variety to break out of local minima. The mutate function finds the distance from the genome to the three genomes we are finding the median for, as well as the estimated median distance. A new genome is then generated on the sorting path between the input genome and whichever genome it's farthest away from the estimated distance of.

In [ ]:
function mutate(genomes, genome)
    # Calculate the DCJ distances between the leaves
    distance12 = dcj_distance(genomes[1], genomes[2])
    distance13 = dcj_distance(genomes[1], genomes[3])
    distance23 = dcj_distance(genomes[2], genomes[3])

    # Calculate estimated distances from the median to the three given genomes
    distance1M = (distance12 + distance13 - distance23) ÷ 2
    distance2M = (distance12 + distance23 - distance13) ÷ 2
    distance3M = (distance13 + distance23 - distance12) ÷ 2

    # Calculate actual distances from genome to the leaves
    dG1 = dcj_distance(genome, genomes[1])
    dG2 = dcj_distance(genome, genomes[2])
    dG3 = dcj_distance(genome, genomes[3])

    # Calculate differences from estimated distances
    differences = [abs(dG1 - distance1M), abs(dG2 - distance2M), abs(dG3 - distance3M)]
    max_diff_index = argmax(differences)

    # Select the target genome for mutation based on the greatest distance
    target_genome = genomes[max_diff_index]

    # Choose a random number of operations for mutation
    m = rand(1:5)

    # Apply m DCJ operations to mutate the genome
    mutated_genome = deepcopy(genome)
    mutated_genome = get_operations(mutated_genome, target_genome, m)

    return mutated_genome
end


mutate (generic function with 1 method)

**Read File:**

Our read file function is designed to work with the input format of our tsv files. The file is read in and preprocessed to drop all irrelevant rows and columns, such as processing out mitochondrial data, which is highly preserved, and accession information, which is not usueful for our purposes.

In [ ]:
import Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
using CSV
using DataFrames

function read_file(filename)
    println(filename)
    data = CSV.File(filename, delim='\t')
    println("read done")
    dataframe = DataFrame(data)
    new_dataframe = copy(dataframe)

    new_dataframe = filter(row -> occursin("protein-coding", row[Symbol("Gene Type")]), new_dataframe)
    new_dataframe = filter(row -> !occursin("uncharacterized gene", row[Symbol("Name")]), new_dataframe)

    # Check if "Chromosome" is missing before calling occursin
    new_dataframe = filter(row -> !ismissing(row[Symbol("Chromosome")]) &&
    !occursin("MT", row[Symbol("Chromosome")]), new_dataframe)

    # Use select! to modify the DataFrame in place rather than creating multiple intermediate copies
    select!(new_dataframe, Not([Symbol("Name"), Symbol("Protein length"), Symbol("Transcripts accession"),
                                Symbol("Protein accession"), Symbol("Accession"), Symbol("Gene ID"),
                                Symbol("Gene Type")]))
    return new_dataframe
end

    Updating registry at `~/opt/anaconda3/envs/csc_530/share/julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/opt/anaconda3/envs/CSC_530/share/julia/environments/CSC_530/Project.toml`
  No Changes to `~/opt/anaconda3/envs/CSC_530/share/julia/environments/CSC_530/Manifest.toml`
   Resolving package versions...
  No Changes to `~/opt/anaconda3/envs/CSC_530/share/julia/environments/CSC_530/Project.toml`
  No Changes to `~/opt/anaconda3/envs/CSC_530/share/julia/environments/CSC_530/Manifest.toml`


read_file (generic function with 1 method)

**Read Genomes:**

This alternative file reading function is used to upload the sample data from the plos paper into our genome representation format for testing purposes.

In [ ]:
function read_genomes(filename)
    genomes = []
    current_genome = []
    current_chromosome = [0]  # Start with a telomere

    open(filename, "r") do file
        for line in eachline(file)
            line = strip(line)
            if startswith(line, ">")  # New genome
                if !isempty(current_chromosome) > 1
                    push!(current_chromosome, 0)  # End with a telomere
                    push!(current_genome, current_chromosome)
                end
                if !isempty(current_genome)
                    push!(genomes, current_genome)
                end
                current_genome = []
                current_chromosome = [0]
            elseif line != ""
                genes = split(line)
                for gene in genes
                    if gene == "\$"
                        push!(current_chromosome, 0)  # End with a telomere
                        if !isempty(current_chromosome)
                            push!(current_genome, current_chromosome)
                        end
                        current_chromosome = [0]  # Start new chromosome with a telomere
                    elseif gene != ""
                        push!(current_chromosome, parse(Int, gene))
                    end
                end
            end
        end

        # Handle the last chromosome and genome
        if length(current_chromosome) > 1
            push!(current_chromosome, 0)  # End with a telomere
            push!(current_genome, current_chromosome)
        end
        if !isempty(current_genome)
            push!(genomes, current_genome)
        end
    end

    return genomes
end


read_genomes (generic function with 1 method)

**Make Dict:**

This function is used to help create the genome representation of our input genome data to use in our genetic algorithm. The function finds all genes that are shared by all three genomes, and then creates a dictionary mapping each one to a number. This allows us to encode a consistent representation across all three genomes.

In [ ]:
function make_dict(df1, df2, df3)
    # Use intersection of sets for efficient lookup and to ensure uniqueness
    symbols_set1 = Set(df1[!, Symbol("Symbol")])
    symbols_set2 = Set(df2[!, Symbol("Symbol")])
    symbols_set3 = Set(df3[!, Symbol("Symbol")])

    # Find common symbols across all three dataframes
    common_symbols = intersect(symbols_set1, symbols_set2, symbols_set3)

    # Create a dictionary from the common symbols with incremental encoding
    dict = Dict{String, Int}([(symbol, index) for (index, symbol) in enumerate(common_symbols)])

    return dict
end


make_dict (generic function with 1 method)

**Encoding:**

This function encodes each of our genomes into our representation format using a pre-computed dictionary mapping genomes to a number. A genome is represented as an array of arrays, with each inner array representing a chromosome. Each value in the chromosome array represents a gene based off the mapping in the dictionary. The gene can be positive for a plus orientation, or negative for a negative orientation. Telomeres are represented using a 0. Chromosome arrays that do not contain any telomeres are circular chromosomes.

In [ ]:
function encoding(df, dict)
    sequences = []  # Array to store chromosomes
    curr_sequence = []  # Current chromosome sequence
    currChro = ""  # Current chromosome identifier

    for row in eachrow(df)
        symbol = row[Symbol("Symbol")]
        chro = row[Symbol("Chromosome")]
        sign = row[Symbol("Orientation")]

        # Check if a new chromosome is encountered
        if chro != currChro
            # Add the current chromosome if it contains more than just a telomere
            if length(curr_sequence) > 1
                push!(curr_sequence, 0)  # Add a telomere to end the chromosome
                push!(sequences, curr_sequence)
            end
            # Start a new chromosome sequence
            curr_sequence = [0]  # Start with a telomere
            currChro = chro
        end

        # Add the gene encoding to the current chromosome
        if haskey(dict, symbol)
            gene_code = dict[symbol]›
            append!(curr_sequence, sign == "plus" ? gene_code : -gene_code)
        end
    end

    # Add the last chromosome if it contains more than just a telomere
    if length(curr_sequence) > 1
        push!(curr_sequence, 0)  # End with a telomere
        push!(sequences, curr_sequence)
    end

    return sequences
end



encoding (generic function with 1 method)

Read in big cat data file

In [ ]:
snow_leopard_file = "snowLeopard_raw.tsv"
tiger_file = "tiger_raw.tsv"
lion_file = "lion_raw.tsv"

sl_df = read_file(snow_leopard_file)
lion_df = read_file(lion_file)
tiger_df = read_file(tiger_file)

snowLeopard_raw.tsv
read done
lion_raw.tsv
read done
tiger_raw.tsv
read done


Row,Begin,End,Chromosome,Orientation,Symbol
,Int64,Int64,String3?,String7,String15?
1,7619,21710,A1,plus,LOC102958199
2,77909,81877,A1,minus,CCDC115
3,81962,86631,A1,plus,IMP4
4,96735,115102,A1,plus,PTPN18
5,167025,169922,A1,plus,LOC122230760
6,170480,188441,A1,plus,LOC122241520
7,188614,200720,A1,plus,LOC122241522
8,201764,224565,A1,plus,LOC102952371
9,305069,333952,A1,plus,ATP12A


In [ ]:
dict = make_dict(sl_df, tiger_df, lion_df)

Dict{String, Int64} with 14740 entries:
  "VPS41"    => 14740
  "SHTN1"    => 7379
  "TMOD4"    => 2
  "ETV4"     => 3
  "IQCF6"    => 7380
  "HOXA9"    => 7381
  "NKAIN3"   => 7382
  "SMTNL2"   => 7383
  "FAM124A"  => 7384
  "IRS2"     => 7385
  "DNASE2"   => 4
  "COLEC11"  => 7386
  "TMSB4X"   => 7387
  "GRM7"     => 7388
  "EIF4EBP3" => 7389
  "KAT2A"    => 5
  "MORC2"    => 7390
  "IL17F"    => 7391
  "ZSWIM7"   => 7392
  "TMEM208"  => 7393
  "SREBF2"   => 6
  "CDC40"    => 7
  "IL12RB2"  => 7394
  "TEX49"    => 8
  "FAXC"     => 9
  ⋮          => ⋮

In [ ]:
encoded_sl = encoding(sl_df, dict)
encoded_tiger = encoding(tiger_df, dict)
encoded_lion = encoding(lion_df, dict)

20-element Vector{Any}:
 [0, -8420, 2258, 5473, 11465, 2465, -11749, 6424, 1202, -9120  …  6479, -11336, 6512, -7052, -11210, -1291, 8629, -12543, -8934, 0]
 [0, -13849, -12534, -9535, -6810, 1569, 1655, 6201, 2649, 12989  …  3338, -6264, 2067, 5282, -7403, -370, -12005, 14087, -13499, 0]
 [0, -6604, -3101, 7031, -5211, 834, 8198, -8572, 6126, -1634  …  6458, 2371, -3657, -10385, 13063, 11852, -1537, 419, 9671, 0]
 [0, -1266, 10783, -7664, 3132, 2282, 4651, 1493, 5425, -4239  …  -13307, 14014, 3031, -11501, 1313, -5091, 4648, -837, -2332, 0]
 [0, -8359, 6945, -1963, -619, 9015, -9569, 3682, 13761, 8997  …  -585, 1466, -2934, -13074, 4697, -5125, 14676, -12636, -8416, 0]
 [0, 13755, -13868, -10079, -3196, -831, -10405, -7505, 14466, -7588  …  -14226, -1365, -991, 7952, 9446, 1543, 235, 1514, 13271, 0]
 [0, 13438, -6248, -12505, 11161, -4675, 14582, -11917, -4928, -12299  …  7871, -11360, -8005, -7949, 8182, -838, 13243, 9436, -7533, 0]
 [0, 9520, -2106, 2455, 4268, -7167, -3934, 3765, 5

This block is used for intial population generation as well as defining parameters. The sample variable contains the genomes from the reference paper, which are stored in the data.txt file. The commented out line defines genomes using the big cat genomes instead. Keep in mind that using the big cat data is significantly slower due to the size of the genomes.

In [ ]:
#genomes = [encoded_sl, encoded_tiger, encoded_lion]
sample = read_genomes("data.txt")
genomes = [sample[1],sample[2],sample[3]]
per_step = 100 #Can be adjusted to control the variance in the initial population
pop = initial_population(genomes, per_step)

max_generations = 25 #Can be adjusted for testing purposes. Reference paper uses 500
best_genome = nothing
top_score = Inf

perfect_median_score = best_score(genomes[1],genomes[2],genomes[3])
println("done")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


This is the main loop of the program. This loop performs the genetic algorithm by creating a new population each iteration by performing crossover and mutations on each genome in the population. When the loop has either completed the maximum number of iterations or converged on the median genome, the highest performing genome is reported, and a plot showing the top score in the population vs iteration number is shown.

In [ ]:
Pkg.add("BenchmarkTools")
using BenchmarkTools
using Plots

top_scores = [] #Used for the plot

println(max_generations)
#benchmark_result = @benchmark
for i in 1:max_generations
        println("Getting scores: ")
        flush(stdout)
        fitness_dict, sorted_genomes = get_scores(genomes, pop, perfect_median_score)

        # Check for the best score in this generation
        current_top_score = fitness_dict[sorted_genomes[1]]
        if current_top_score < top_score
            top_score = current_top_score
            best_genome = sorted_genomes[1]
        end

        push!(top_scores, top_score)

        # Early termination if perfect median score is met
        if top_score == perfect_median_score  # Define 'perfect_median_score' as per your criteria
            println("Perfect median score met at generation $i")
            break
        end

        new_pop = []

        #Select the top 10% of individuals
        top_10_percent = ceil(Int, length(sorted_genomes) * 0.1)
        new_pop = [deepcopy(genome) for genome in sorted_genomes[1:top_10_percent]]

        # Selection for crossover from the remaining 90%
        for j in 1:((length(sorted_genomes) - top_10_percent))
            parent1 = sorted_genomes[rand(top_10_percent+1:end)]
            parent2 = sorted_genomes[rand(top_10_percent+1:end)]
            println("crossover")
            C1, C2 = crossover(parent1, parent2, genomes..., perfect_median_score)

            # Mutation
            println("mutate")
            C1s = mutate(genomes, C1)
            C2s = mutate(genomes, C2)

            # Evaluate fitness for each child
            fitness_scores = [
                (C1, fitness(genomes[1], genomes[2], genomes[3], C1, perfect_median_score)),
                (C2, fitness(genomes[1], genomes[2], genomes[3], C2, perfect_median_score)),
                (C1s, fitness(genomes[1], genomes[2], genomes[3], C1s, perfect_median_score)),
                (C2s, fitness(genomes[1], genomes[2], genomes[3], C2s, perfect_median_score))
            ]


            # Sort by fitness and select the top two
            best_two = sort(fitness_scores, by = x -> x[2], rev = true)[1:2]

            # Add the best two offspring to new population
            push!(new_pop, best_two[1][1])
            push!(new_pop, best_two[2][1])

            println(j)
        end

        # Replace the old population with the new one
        pop = new_pop
        println(i)
        flush(stdout)
end #samples=1 evals=1

#println(benchmark_result)


println("Best genome found: $best_genome with score: $top_score")
plot(1:max_generations, top_scores, title="Fitness Score vs Iteration", xlabel="Iteration", ylabel="Fitness Score", legend=false)

   Resolving package versions...
  No Changes to `~/opt/anaconda3/envs/CSC_530/share/julia/environments/CSC_530/Project.toml`
  No Changes to `~/opt/anaconda3/envs/CSC_530/share/julia/environments/CSC_530/Manifest.toml`


25
Getting scores: 
---------------------

LoadError: InterruptException:

This block defines a dictionary mapping and a function for the teaching staff to be able to convert the standardized genome format into our genome format for testing purposes.

In [ ]:
# Global dictionary for character to gene mapping
global_map = Dict(
    'a' => 1, 'b' => 2, 'c' => 3, 'd' => 4, 'e' => 5, 'f' => 6, 'g' => 7, 'h' => 8, 'i' => 9, 'j' => 10,
    'k' => 11, 'l' => 12, 'm' => 13, 'n' => 14, 'o' => 15, 'p' => 16, 'q' => 17, 'r' => 18, 's' => 19, 't' => 20,
    'u' => 21, 'v' => 22, 'w' => 23, 'x' => 24, 'y' => 25, 'z' => 26,
    'A' => -1, 'B' => -2, 'C' => -3, 'D' => -4, 'E' => -5, 'F' => -6, 'G' => -7, 'H' => -8, 'I' => -9, 'J' => -10,
    'K' => -11, 'L' => -12, 'M' => -13, 'N' => -14, 'O' => -15, 'P' => -16, 'Q' => -17, 'R' => -18, 'S' => -19, 'T' => -20,
    'U' => -21, 'V' => -22, 'W' => -23, 'X' => -24, 'Y' => -25, 'Z' => -26
)
function convert_to_genome(input)
    # Function to convert a character to its gene number
    function char_to_int(c)
        if c == '.'
            return 0
        else
            return global_map[c]
        end
    end

    # Convert input to genome format
    genome = []
    for chromosome in input
        push!(genome, [char_to_int(c) for c in chromosome])
    end

    return genome
end

# Example usage
input1 = [".abc."]
input2 = ["abC", ".fgH.", "yZ"]
genome1 = convert_to_genome(input1)
genome2 = convert_to_genome(input2)

println(genome1)
println(genome2)


Any[[0, 1, 2, 3, 0]]
Any[[1, 2, -3], [0, 6, 7, -8, 0], [25, -26]]


Verification of our DCJ distance using the example inputs given on Piaza

In [ ]:
A = convert_to_genome([".ab."])
B = convert_to_genome(["ab"])
println(dcj_distance(A, B))

A = convert_to_genome(["ab", ".cd."])
B = convert_to_genome([".abcd."])
println(dcj_distance(A, B))

A = convert_to_genome([".abC.", "de"])
B = convert_to_genome([".abdCe."])
println(dcj_distance(A, B))

1
1
3
